In [22]:
import azureml.core

In [23]:
print(azureml.core.VERSION)

1.17.0


# Create Workspace

In [24]:
from azureml.core import Workspace
from azureml.core.authentication import InteractiveLoginAuthentication

sid = '9611813b-7660-4002-96a4-dec673f84339'
tenant_id = '0f942ca0-ebef-4f26-80f8-f501d599ba90'

forced_interactive_auth = InteractiveLoginAuthentication(tenant_id=tenant_id, force=True)

ws = Workspace.create(name='azureml_workspace3',
            subscription_id= sid, 
            resource_group='rgazureml',
            create_resource_group = True,
            location='centralus'
            )

Performing interactive authentication. Please follow the instructions on the terminal.


WARNING - Note, we have launched a browser for you to login. For old experience with device code, use "az login --use-device-code"


You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.


Deploying StorageAccount with name azuremlwstorage963f5f332.
Deploying KeyVault with name azuremlwkeyvaultc78f4d9c.
Deploying AppInsights with name azuremlwinsights763d713b.
Deployed AppInsights with name azuremlwinsights763d713b. Took 7.61 seconds.
Deployed KeyVault with name azuremlwkeyvaultc78f4d9c. Took 25.58 seconds.
Deployed StorageAccount with name azuremlwstorage963f5f332. Took 31.92 seconds.
Deploying Workspace with name azureml_workspace3.
Deployed Workspace with name azureml_workspace3. Took 30.34 seconds.


# Upload to Default Data Store

In [25]:
#upload data by using get_default_datastore()
ds = ws.get_default_datastore()
ds.upload(src_dir='./winedata', target_path='winedata', overwrite=True, show_progress=True)

print('Done')

Uploading an estimated of 2 files
Uploading ./winedata\winequality_red.csv
Uploaded ./winedata\winequality_red.csv, 1 files out of an estimated total of 2
Uploading ./winedata\wine.csv
Uploaded ./winedata\wine.csv, 2 files out of an estimated total of 2
Uploaded 2 files
Done


# Create the Training Folder

In [26]:
import os

# create the folder
folder_training_script = './winecode'
os.makedirs(folder_training_script, exist_ok=True)

print('Done')

Done


# Create the Compute Cluster

In [27]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
import os

# Step 1: name the cluster and set the minimal and maximal number of nodes 
compute_name = os.environ.get("AML_COMPUTE_CLUSTER_NAME", "cpucluster")
min_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MIN_NODES", 0)
max_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MAX_NODES", 1)

# Step 2: choose environment variables 
vm_size = os.environ.get("AML_COMPUTE_CLUSTER_SKU", "STANDARD_D2_V2")

provisioning_config = AmlCompute.provisioning_configuration(
    vm_size = vm_size, min_nodes = min_nodes, max_nodes = max_nodes)

# create the cluster
compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)

print('Compute target created')

Compute target created


# Create the Training Script

In [28]:
%%writefile $folder_training_script/train.py

import argparse
import os
import numpy as np
import pandas as pd
import glob

from azureml.core import Run
# from utils import load_data

# let user feed in 2 parameters, the dataset to mount or download, and the regularization rate of the logistic regression model
parser = argparse.ArgumentParser()
parser.add_argument('--data-folder', type=str, dest='data_folder', help='data folder mounting point')
args = parser.parse_args()

###
data_folder = os.path.join(args.data_folder, 'winedata')
print('Data folder:', data_folder)

red_wine = pd.read_csv(os.path.join(data_folder, 'winequality_red.csv'))

                        
from tensorflow import keras
from tensorflow.keras import layers, callbacks


# Create training and validation splits
df_train = red_wine.sample(frac=0.7, random_state=0)
df_valid = red_wine.drop(df_train.index)

X = df_train.copy()
X = X.drop(columns = ["quality"])
df_train_stats = X.describe()
df_train_stats = df_train_stats.transpose()

def norm(x):
    return (x - df_train_stats['mean']) / df_train_stats['std']

# Split features and target
X_train = df_train.drop('quality', axis=1)
X_valid = df_valid.drop('quality', axis=1)
X_train = norm(X_train)
X_valid = norm(X_valid)
y_train = df_train['quality']
y_valid = df_valid['quality']


early_stopping = callbacks.EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=20, # how many epochs to wait before stopping
    restore_best_weights=True,
)

input_shape=X_train.shape[1]

model = keras.Sequential([
    # the hidden ReLU layers
    layers.Dense(units=512, activation='relu', input_shape=[input_shape]),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    layers.Dense(units=512, activation='relu'),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    layers.Dense(units=512, activation='relu'),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    # the linear output layer 
    layers.Dense(units=1)
])

model.compile(
    optimizer='adam',
    loss='mae',
)

history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    batch_size=256,
    epochs=500,
    callbacks=[early_stopping], # put your callbacks in a list
    verbose=0,  # turn off training log
)

history_df = pd.DataFrame(history.history)

# Get the experiment run context
run = Run.get_context()

run.log('min_val_loss', np.float(history_df['val_loss'].min()))

os.makedirs('outputs', exist_ok=True)

# note file saved in the outputs folder is automatically uploaded into experiment record
model.save('outputs/my_model')

run.complete()

Overwriting ./winecode/train.py


# Create the Environment

In [29]:
from azureml.core import Environment
curated_env_name = 'AzureML-TensorFlow-2.2-GPU'
tf_env = Environment.get(workspace=ws, name=curated_env_name)

# Register the environment in the workspace

# Create the Estimator

In [30]:
from azureml.train.estimator import Estimator

script_params = {
    '--data-folder': ds.as_mount()
}


# Create an estimator
estimator = Estimator(source_directory=folder_training_script,
                      script_params=script_params,
                      compute_target = compute_target, # Run the experiment on the remote compute target
                      environment_definition = tf_env,
                      entry_script='train.py')

# Create the Experiment

In [31]:
from azureml.core import Experiment

#Create an experiment
experiment = Experiment(workspace = ws, name = "wine_expt")

print('Experiment created')

Experiment created


# Submit the Experiment with the Estimator information

In [32]:
run = experiment.submit(config=estimator)
run

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.
WARNING - If 'arguments' has been provided here and arguments have been specified in 'run_config', 'arguments' provided in ScriptRunConfig initialization will take precedence.


Experiment,Id,Type,Status,Details Page,Docs Page
wine_expt,wine_expt_1619178650_0ee9c8ec,azureml.scriptrun,Preparing,Link to Azure Machine Learning studio,Link to Documentation


In [34]:
#get the result
print(run.get_metrics())

{'min_val_loss': 0.5298544764518738}


# Register the Model

In [35]:
model = run.register_model(model_name='wine_model',
                           model_path='outputs/my_model',
                           tags = {'area': "wine", 'type': "sklearn"},
                           description = "quality wine")

print(model.name, model.id, model.version, sep='\t')

wine_model	wine_model:1	1


# Inference Section

## Create the Inference Script

In [36]:
%%writefile $folder_training_script/score.py

import json
from tensorflow import keras
import numpy as np
from azureml.core.model import Model

# Called when the service is loaded
def init():
    global model
    # Get the path to the registered model file and load it
    model_path = Model.get_model_path('wine_model')
    model = keras.models.load_model(model_path)

# Called when a request is received
def run(raw_data):
    try:
               
        # Get the input data as a numpy array
        data = np.array(json.loads(raw_data)['data']) 
        # Get a prediction from the model
        predictions = model.predict(data)
        log_txt = 'Data:' + str(data) + ' - Predictions:' + str(predictions)
        print(log_txt)
        # Return the predictions as any JSON serializable format
        return predictions.tolist()
    
    except Exception as e:
        result = str(e)
        # return error message back to the client
        return json.dumps({"error": result})

Overwriting ./winecode/score.py


#  Create the Dependencies for the Inference Script

# Create an Inference Configuration

In [37]:
from azureml.core.model import InferenceConfig

classifier_inference_config = InferenceConfig(source_directory = './winecode',
                                              entry_script="score.py",
                                               environment=tf_env)

# Create the Inference Cluster

In [38]:
from azureml.core.compute import ComputeTarget, AksCompute

cluster_name = 'aks-cluster'
compute_config = AksCompute.provisioning_configuration(cluster_purpose = AksCompute.ClusterPurpose.DEV_TEST)
production_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
production_cluster.wait_for_completion(show_output=True)

Creating.....................................................
SucceededProvisioning operation finished, operation "Succeeded"


In [39]:
from azureml.core.webservice import AksWebservice

classifier_deploy_config = AksWebservice.deploy_configuration(cpu_cores = 1,
                                                              memory_gb = 1)

# Deploy the Model to the Inference Cluster

In [40]:
from azureml.core.model import Model

model = ws.models['wine_model']
service = Model.deploy(workspace=ws,
                       name = 'wine-service0003',
                       models = [model],
                       inference_config = classifier_inference_config,
                       deployment_config = classifier_deploy_config,
                       deployment_target = production_cluster)
service.wait_for_deployment(show_output = True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running...........................................................................................
Succeeded
AKS service creation operation finished, operation "Succeeded"


In [41]:
service.update(enable_app_insights=True)

# Perform predictions with the Inference Cluster

In [42]:
endpoint = service.scoring_uri
print(endpoint)

http://52.154.204.53:80/api/v1/service/wine-service0003/score


In [43]:
primary_key, secondary_key = service.get_keys()

In [44]:
primary_key

'IRgI7Wopg220fYlQlTwAaqs0FMgHnZSx'

In [48]:
import requests
import json

# An array of new data cases
x_new = [[7.3,0.65,0,1.2,0.065,15,21,0.9946,3.39,0.47,10]]

# Convert the array to a serializable list in a JSON document
json_data = json.dumps({"data": x_new})

# Set the content type in the request headers
request_headers = { "Content-Type":"application/json",
                    "Authorization":"Bearer " + primary_key }

# Call the service
response = requests.post(url = endpoint,
                         data = json_data,
                         headers = request_headers)

print(response)

<Response [200]>


In [49]:
response.json()

[[4.49352502822876]]